In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from pandas_datareader import data
import quandl
import yfinance as yf
import pandas_datareader.data as web
import datetime
from pandas_profiling import ProfileReport

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

To-Do's

1. Add TRIN, NYSI
2. Stochastic
3. RSI
4. TSI
5. Bollinger Bands
6. Candlesticks
7. % above/below MA
8. Put/Call ratio and volume
# dispersion_20d                         (0.4689065622873918)
# market_vol_120d                        (0.19307524853086544)

In [2]:
start = datetime.datetime(2002,12,30)
fedhold = web.DataReader('WSHOSHO', 'fred', start)
fedbal = web.DataReader('WALCL', 'fred', start)
stres = web.DataReader('STLFSI', 'fred', start)
freight = web.DataReader('FRGSHPUSM649NCIS', 'fred', start)
leading = web.DataReader('USALOLITONOSTSAM', 'fred', start)
hyspread = web.DataReader('BAMLH0A0HYM2', 'fred', start)
tentwospr = web.DataReader('T10Y2Y', 'fred', start)
fedrecprob = web.DataReader('RECPROUSM156N', 'fred', start)

In [3]:
print('Size of Fed Treasury Holdings', fedhold.shape)
print('Size of Fed Balance Sheet', fedbal.shape)
print('Size of Stress Index', stres.shape)
print('Size of Freight data', freight.shape)
print('Size of LEI data', leading.shape)
print('Size of High Yield Spread', hyspread.shape)
print('Size of 10yr Treasury vs Two year Treasury Spread', tentwospr.shape)
print('Size of Fed Recession Probability', fedrecprob.shape),

Size of Fed Treasury Holdings (893, 1)
Size of Fed Balance Sheet (893, 1)
Size of Stress Index (892, 1)
Size of Freight data (204, 1)
Size of LEI data (203, 1)
Size of High Yield Spread (4522, 1)
Size of 10yr Treasury vs Two year Treasury Spread (4466, 1)
Size of Fed Recession Probability (204, 1)


(None,)

In [8]:
leading.head()

,USALOLITONOSTSAM
DATE,
2003-01-01,97.874659
2003-02-01,97.846193
2003-03-01,97.889520
2003-04-01,98.027059
2003-05-01,98.249626


In [9]:
tnx = yf.download( "^TNX", start="2003-02-28", end="2020-02-11",auto_adjust = True)
spy = yf.download( "SPY", start="2003-02-28", end="2020-02-11",auto_adjust = True)
vix = yf.download('^VIX', start="2003-02-28", end="2020-02-11",auto_adjust = True)
tlt = yf.download('TLT', start="2003-02-28", end="2020-02-11",auto_adjust = True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
spy.head()

,Open,High,Low,Close,Volume
Date,,,,,
2003-02-28,60.370961,60.914137,60.149405,60.678284,43666800
2003-03-03,60.935599,61.307242,59.834955,60.099392,42923100
2003-03-04,59.999304,60.042190,59.070193,59.141663,31440500
2003-03-05,59.041627,59.706302,58.862952,59.641975,43974700
2003-03-06,59.227429,59.691981,58.941547,59.141663,41217700


In [11]:
spy['1d%Chg']=spy['Close'].pct_change()
spy['2d%Chg']=spy['Close'].pct_change(2)
spy['3d%Chg']=spy['Close'].pct_change(3)
spy['5d%Chg']=spy['Close'].pct_change(5)
spy['7d%Chg']=spy['Close'].pct_change(7)
spy['10d%Chg']=spy['Close'].pct_change(10)
spy['14d%Chg']=spy['Close'].pct_change(14)
spy['21d%Chg']=spy['Close'].pct_change(21)
spy['30d%Chg']=spy['Close'].pct_change(30)
spy['45d%Chg']=spy['Close'].pct_change(45)
spy['60d%Chg']=spy['Close'].pct_change(60)
spy['90d%Chg']=spy['Close'].pct_change(90)
spy['120d%Chg']=spy['Close'].pct_change(120)
spy['180d%Chg']=spy['Close'].pct_change(180)
spy['252d%Chg']=spy['Close'].pct_change(252)





In [12]:
spy['200day_MA'] = spy['Close'].rolling(window=200).mean()
spy['150day_MA'] = spy['Close'].rolling(window=150).mean()
spy['100day_MA'] = spy['Close'].rolling(window=100).mean()
spy['50day_MA'] = spy['Close'].rolling(window=50).mean()
spy['25day_MA'] = spy['Close'].rolling(window=25).mean()
spy['13day_MA'] = spy['Close'].rolling(window=13).mean()
spy['8day_MA'] = spy['Close'].rolling(window=8).mean()
spy['5day_MA'] = spy['Close'].rolling(window=5).mean()

In [13]:
spy['20stdev']=spy['Close'].rolling(20).std()
spy['bbwidth']=spy['20stdev']*4
spy['20dMA']= spy['Close'].rolling(20).mean()
spy['bb%']= (spy['Close']- spy['20dMA'] + 2 * spy['20stdev'])/(4 * spy['20stdev'])
spy['upbb']= spy['20dMA']+2*spy['20stdev']
spy['lwbb']= spy['20dMA']-2*spy['20stdev']

In [14]:
spy.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', '1d%Chg', '2d%Chg', '3d%Chg',
       '5d%Chg', '7d%Chg', '10d%Chg', '14d%Chg', '21d%Chg', '30d%Chg',
       '45d%Chg', '60d%Chg', '90d%Chg', '120d%Chg', '180d%Chg', '252d%Chg',
       '200day_MA', '150day_MA', '100day_MA', '50day_MA', '25day_MA',
       '13day_MA', '8day_MA', '5day_MA', '20stdev', 'bbwidth', '20dMA', 'bb%',
       'upbb', 'lwbb'],
      dtype='object')

In [15]:
spy.columns=['SPY_'+i for i in spy.columns]


In [16]:
spy.columns

Index(['SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume',
       'SPY_1d%Chg', 'SPY_2d%Chg', 'SPY_3d%Chg', 'SPY_5d%Chg', 'SPY_7d%Chg',
       'SPY_10d%Chg', 'SPY_14d%Chg', 'SPY_21d%Chg', 'SPY_30d%Chg',
       'SPY_45d%Chg', 'SPY_60d%Chg', 'SPY_90d%Chg', 'SPY_120d%Chg',
       'SPY_180d%Chg', 'SPY_252d%Chg', 'SPY_200day_MA', 'SPY_150day_MA',
       'SPY_100day_MA', 'SPY_50day_MA', 'SPY_25day_MA', 'SPY_13day_MA',
       'SPY_8day_MA', 'SPY_5day_MA', 'SPY_20stdev', 'SPY_bbwidth', 'SPY_20dMA',
       'SPY_bb%', 'SPY_upbb', 'SPY_lwbb'],
      dtype='object')

In [17]:
tnx['1d%Chg']=tnx['Close'].pct_change()
tnx['2d%Chg']=tnx['Close'].pct_change(2)
tnx['3d%Chg']=tnx['Close'].pct_change(3)
tnx['5d%Chg']=tnx['Close'].pct_change(5)
tnx['7d%Chg']=tnx['Close'].pct_change(7)
tnx['10d%Chg']=tnx['Close'].pct_change(10)
tnx['14d%Chg']=tnx['Close'].pct_change(14)
tnx['21d%Chg']=tnx['Close'].pct_change(21)
tnx['30d%Chg']=tnx['Close'].pct_change(30)
tnx['45d%Chg']=tnx['Close'].pct_change(45)
tnx['60d%Chg']=tnx['Close'].pct_change(60)
tnx['90d%Chg']=tnx['Close'].pct_change(90)
tnx['120d%Chg']=tnx['Close'].pct_change(120)
tnx['180d%Chg']=tnx['Close'].pct_change(180)
tnx['252d%Chg']=tnx['Close'].pct_change(252)





In [18]:
tnx['200day_MA'] = tnx['Close'].rolling(window=200).mean()
tnx['150day_MA'] = tnx['Close'].rolling(window=150).mean()
tnx['100day_MA'] = tnx['Close'].rolling(window=100).mean()
tnx['50day_MA'] = tnx['Close'].rolling(window=50).mean()
tnx['25day_MA'] = tnx['Close'].rolling(window=25).mean()
tnx['13day_MA'] = tnx['Close'].rolling(window=13).mean()
tnx['8day_MA'] = tnx['Close'].rolling(window=8).mean()
tnx['5day_MA'] = tnx['Close'].rolling(window=5).mean()

In [19]:
tnx.columns=['TNX_'+i for i in tnx.columns]
tnx.drop(['TNX_Volume'], axis=1, inplace=True)

In [20]:
vix['1d%Chg']=vix['Close'].pct_change()
vix['2d%Chg']=vix['Close'].pct_change(2)
vix['3d%Chg']=vix['Close'].pct_change(3)
vix['5d%Chg']=vix['Close'].pct_change(5)
vix['7d%Chg']=vix['Close'].pct_change(7)
vix['10d%Chg']=vix['Close'].pct_change(10)
vix['14d%Chg']=vix['Close'].pct_change(14)
vix['21d%Chg']=vix['Close'].pct_change(21)
vix['30d%Chg']=vix['Close'].pct_change(30)
vix['45d%Chg']=vix['Close'].pct_change(45)
vix['60d%Chg']=vix['Close'].pct_change(60)
vix['90d%Chg']=vix['Close'].pct_change(90)
vix['120d%Chg']=vix['Close'].pct_change(120)
vix['180d%Chg']=vix['Close'].pct_change(180)
vix['252d%Chg']=vix['Close'].pct_change(252)

In [21]:
vix['200day_MA'] = vix['Close'].rolling(window=200).mean()
vix['150day_MA'] = vix['Close'].rolling(window=150).mean()
vix['100day_MA'] = vix['Close'].rolling(window=100).mean()
vix['50day_MA'] = vix['Close'].rolling(window=50).mean()
vix['25day_MA'] = vix['Close'].rolling(window=25).mean()
vix['13day_MA'] = vix['Close'].rolling(window=13).mean()
vix['8day_MA'] = vix['Close'].rolling(window=8).mean()
vix['5day_MA'] = vix['Close'].rolling(window=5).mean()

In [22]:
vix['20stdev']=vix['Close'].rolling(20).std()
vix['bbwidth']=vix['20stdev']*4
vix['20dMA']= vix['Close'].rolling(20).mean()
vix['bb%']= (vix['Close']- vix['20dMA'] + 2 * vix['20stdev'])/(4 * vix['20stdev'])
vix['upbb']= vix['20dMA']+2*vix['20stdev']
vix['lwbb']= vix['20dMA']-2*vix['20stdev']

In [23]:
vix.columns=['VIX_'+i for i in vix.columns]
vix.drop(['VIX_Volume'], axis=1, inplace=True)

In [24]:
tlt['1d%Chg']=tlt['Close'].pct_change()
tlt['2d%Chg']=tlt['Close'].pct_change(2)
tlt['3d%Chg']=tlt['Close'].pct_change(3)
tlt['5d%Chg']=tlt['Close'].pct_change(5)
tlt['7d%Chg']=tlt['Close'].pct_change(7)
tlt['10d%Chg']=tlt['Close'].pct_change(10)
tlt['14d%Chg']=tlt['Close'].pct_change(14)
tlt['21d%Chg']=tlt['Close'].pct_change(21)
tlt['30d%Chg']=tlt['Close'].pct_change(30)
tlt['45d%Chg']=tlt['Close'].pct_change(45)
tlt['60d%Chg']=tlt['Close'].pct_change(60)
tlt['90d%Chg']=tlt['Close'].pct_change(90)
tlt['120d%Chg']=tlt['Close'].pct_change(120)
tlt['180d%Chg']=tlt['Close'].pct_change(180)
tlt['252d%Chg']=tlt['Close'].pct_change(252)

In [25]:
tlt['200day_MA'] = tlt['Close'].rolling(window=200).mean()
tlt['150day_MA'] = tlt['Close'].rolling(window=150).mean()
tlt['100day_MA'] = tlt['Close'].rolling(window=100).mean()
tlt['50day_MA'] = tlt['Close'].rolling(window=50).mean()
tlt['25day_MA'] = tlt['Close'].rolling(window=25).mean()
tlt['13day_MA'] = tlt['Close'].rolling(window=13).mean()
tlt['8day_MA'] = tlt['Close'].rolling(window=8).mean()
tlt['5day_MA'] = tlt['Close'].rolling(window=5).mean()

In [26]:
tlt['20stdev']=tlt['Close'].rolling(20).std()
tlt['bbwidth']=tlt['20stdev']*4
tlt['20dMA']= tlt['Close'].rolling(20).mean()
tlt['bb%']= (tlt['Close']- tlt['20dMA'] + 2 * tlt['20stdev'])/(4 * tlt['20stdev'])
tlt['upbb']= tlt['20dMA']+2*tlt['20stdev']
tlt['lwbb']= tlt['20dMA']-2*tlt['20stdev']

In [27]:
tlt.columns=['TLT_'+i for i in tlt.columns]

In [28]:
tlt.head()

,TLT_Open,TLT_High,TLT_Low,TLT_Close,TLT_Volume,TLT_1d%Chg,TLT_2d%Chg,TLT_3d%Chg,TLT_5d%Chg,TLT_7d%Chg,...,TLT_25day_MA,TLT_13day_MA,TLT_8day_MA,TLT_5day_MA,TLT_20stdev,TLT_bbwidth,TLT_20dMA,TLT_bb%,TLT_upbb,TLT_lwbb
Date,,,,,,,,,,,,,,,,,,,,,
2003-02-28,49.010643,49.480638,48.955993,49.442383,213100,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-03,49.371092,49.617945,49.321722,49.502747,204100,0.001221,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-04,49.442381,49.617922,49.382038,49.535641,83100,0.000664,0.001886,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-05,49.711219,49.716701,49.393048,49.700245,126000,0.003323,0.003990,0.005215,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-06,49.672816,49.744128,49.404014,49.508244,166000,-0.003863,-0.000553,0.000111,NaN,NaN,...,NaN,NaN,NaN,49.537852,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
mkt_data = spy.join(tlt).join(vix).join(tnx)
mkt_data.index.name='Date'
mkt_data.tail()

,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,SPY_7d%Chg,...,TNX_180d%Chg,TNX_252d%Chg,TNX_200day_MA,TNX_150day_MA,TNX_100day_MA,TNX_50day_MA,TNX_25day_MA,TNX_13day_MA,TNX_8day_MA,TNX_5day_MA
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-04,328.070007,330.010010,327.720001,329.059998,62573200,0.015241,0.022783,0.004211,0.006638,0.000882,...,-0.337329,-0.405195,1.916920,1.785173,1.77134,1.78992,1.75112,1.665000,1.590250,1.5590
2020-02-05,332.269989,333.089996,330.670013,332.859985,65951100,0.011548,0.026965,0.034594,0.019105,0.028933,...,-0.306852,-0.374194,1.912205,1.782800,1.77081,1.78718,1.74128,1.652692,1.586250,1.5700
2020-02-06,333.910004,334.190002,332.799988,333.980011,50359700,0.003365,0.014952,0.030421,0.019226,0.021689,...,-0.316424,-0.389075,1.907465,1.780427,1.76992,1.78530,1.73028,1.637923,1.591125,1.5872
2020-02-07,332.820007,333.989990,331.600006,332.200012,64088400,-0.005330,-0.001983,0.009542,0.032543,0.017084,...,-0.340577,-0.420705,1.902555,1.777387,1.76779,1.78142,1.71812,1.623231,1.583250,1.5988
2020-02-10,331.230011,334.750000,331.190002,334.679993,36449231,0.007465,0.002096,0.005468,0.032581,0.021362,...,-0.359685,-0.427461,1.897340,1.774527,1.76423,1.77688,1.70848,1.606154,1.577375,1.6042


In [30]:
mkt_data[mkt_data.index.duplicated()]

,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,SPY_7d%Chg,...,TNX_180d%Chg,TNX_252d%Chg,TNX_200day_MA,TNX_150day_MA,TNX_100day_MA,TNX_50day_MA,TNX_25day_MA,TNX_13day_MA,TNX_8day_MA,TNX_5day_MA
Date,,,,,,,,,,,,,,,,,,,,,


In [31]:
mkt_data['TNX_Close'].isna()

Date
2003-02-28    False
2003-03-03    False
2003-03-04    False
2003-03-05    False
2003-03-06    False
              ...  
2020-02-04    False
2020-02-05    False
2020-02-06    False
2020-02-07    False
2020-02-10    False
Name: TNX_Close, Length: 4267, dtype: bool

In [32]:
mkt_data.ffill(axis=0, inplace=True)

In [33]:
mkt_data['stkbdDiff']=mkt_data['SPY_1d%Chg']-mkt_data['TLT_1d%Chg']
mkt_data['stkbdDiff3d']=mkt_data['SPY_3d%Chg']-mkt_data['TLT_3d%Chg']
mkt_data['stkbdDiff5d']=mkt_data['SPY_5d%Chg']-mkt_data['TLT_5d%Chg']
mkt_data['stkbdDiff7d']=mkt_data['SPY_7d%Chg']-mkt_data['TLT_7d%Chg']
mkt_data['stkbdDiff10d']=mkt_data['SPY_10d%Chg']-mkt_data['TLT_10d%Chg']
mkt_data['stkbdDiff14d']=mkt_data['SPY_14d%Chg']-mkt_data['TLT_14d%Chg']
mkt_data['stkbdDiff21d']=mkt_data['SPY_21d%Chg']-mkt_data['TLT_21d%Chg']
mkt_data['stkbdDiff30d']=mkt_data['SPY_30d%Chg']-mkt_data['TLT_30d%Chg']
mkt_data['stkbdDiff45d']=mkt_data['SPY_45d%Chg']-mkt_data['TLT_45d%Chg']
mkt_data['stkbdDiff60d']=mkt_data['SPY_60d%Chg']-mkt_data['TLT_60d%Chg']
mkt_data['stkbdDiff90d']=mkt_data['SPY_90d%Chg']-mkt_data['TLT_90d%Chg']
mkt_data['stkbdDiff120d']=mkt_data['SPY_120d%Chg']-mkt_data['TLT_120d%Chg']
mkt_data['stkbdDiff180d']=mkt_data['SPY_180d%Chg']-mkt_data['TLT_180d%Chg']
mkt_data['stkbdDiff252d']=mkt_data['SPY_252d%Chg']-mkt_data['TLT_252d%Chg']

mkt_data['diff5day_MA'] = mkt_data['stkbdDiff'].rolling(window=5).mean()
mkt_data['diff8day_MA'] = mkt_data['stkbdDiff'].rolling(window=8).mean()
mkt_data['diff13day_MA'] = mkt_data['stkbdDiff'].rolling(window=13).mean()
mkt_data['diff25day_MA'] = mkt_data['stkbdDiff'].rolling(window=25).mean()
mkt_data['diff50day_MA'] = mkt_data['stkbdDiff'].rolling(window=50).mean()
mkt_data['diff100day_MA'] = mkt_data['stkbdDiff'].rolling(window=100).mean()
mkt_data['diff150day_MA'] = mkt_data['stkbdDiff'].rolling(window=150).mean()
mkt_data['diff200day_MA'] = mkt_data['stkbdDiff'].rolling(window=200).mean()

In [34]:
mkt_data.columns.tolist()

['SPY_Open',
 'SPY_High',
 'SPY_Low',
 'SPY_Close',
 'SPY_Volume',
 'SPY_1d%Chg',
 'SPY_2d%Chg',
 'SPY_3d%Chg',
 'SPY_5d%Chg',
 'SPY_7d%Chg',
 'SPY_10d%Chg',
 'SPY_14d%Chg',
 'SPY_21d%Chg',
 'SPY_30d%Chg',
 'SPY_45d%Chg',
 'SPY_60d%Chg',
 'SPY_90d%Chg',
 'SPY_120d%Chg',
 'SPY_180d%Chg',
 'SPY_252d%Chg',
 'SPY_200day_MA',
 'SPY_150day_MA',
 'SPY_100day_MA',
 'SPY_50day_MA',
 'SPY_25day_MA',
 'SPY_13day_MA',
 'SPY_8day_MA',
 'SPY_5day_MA',
 'SPY_20stdev',
 'SPY_bbwidth',
 'SPY_20dMA',
 'SPY_bb%',
 'SPY_upbb',
 'SPY_lwbb',
 'TLT_Open',
 'TLT_High',
 'TLT_Low',
 'TLT_Close',
 'TLT_Volume',
 'TLT_1d%Chg',
 'TLT_2d%Chg',
 'TLT_3d%Chg',
 'TLT_5d%Chg',
 'TLT_7d%Chg',
 'TLT_10d%Chg',
 'TLT_14d%Chg',
 'TLT_21d%Chg',
 'TLT_30d%Chg',
 'TLT_45d%Chg',
 'TLT_60d%Chg',
 'TLT_90d%Chg',
 'TLT_120d%Chg',
 'TLT_180d%Chg',
 'TLT_252d%Chg',
 'TLT_200day_MA',
 'TLT_150day_MA',
 'TLT_100day_MA',
 'TLT_50day_MA',
 'TLT_25day_MA',
 'TLT_13day_MA',
 'TLT_8day_MA',
 'TLT_5day_MA',
 'TLT_20stdev',
 'TLT_bbwidth',


In [35]:
# Define VIX/TNX ratio.
# Dividing VIX index,by 10 year Treasury yield

mkt_data['VIXTNX']=mkt_data['VIX_Close']/mkt_data['TNX_Close']/10


mkt_data['VIXTNX1d%Chg']=mkt_data['VIXTNX'].pct_change()
mkt_data['VIXTNX2d%Chg']=mkt_data['VIXTNX'].pct_change(2)
mkt_data['VIXTNX3d%Chg']=mkt_data['VIXTNX'].pct_change(3)
mkt_data['VIXTNX5d%Chg']=mkt_data['VIXTNX'].pct_change(5)
mkt_data['VIXTNX7d%Chg']=mkt_data['VIXTNX'].pct_change(7)
mkt_data['VIXTNX10d%Chg']=mkt_data['VIXTNX'].pct_change(10)
mkt_data['VIXTNX14d%Chg']=mkt_data['VIXTNX'].pct_change(14)
mkt_data['VIXTNX21d%Chg']=mkt_data['VIXTNX'].pct_change(21)
mkt_data['VIXTNX30d%Chg']=mkt_data['VIXTNX'].pct_change(30)
mkt_data['VIXTNX60d%Chg']=mkt_data['VIXTNX'].pct_change(45)
mkt_data['VIXTNX90d%Chg']=mkt_data['VIXTNX'].pct_change(60)
mkt_data['VIXTNX120d%Chg']=mkt_data['VIXTNX'].pct_change(90)
mkt_data['VIXTNX180d%Chg']=mkt_data['VIXTNX'].pct_change(120)
mkt_data['VIXTNX252d%Chg']=mkt_data['VIXTNX'].pct_change(252)


mkt_data['VIXTNX5day_MA'] = mkt_data['VIXTNX'].rolling(window=5).mean()
mkt_data['VIXTNX8day_MA'] = mkt_data['VIXTNX'].rolling(window=8).mean()
mkt_data['VIXTNX13day_MA'] = mkt_data['VIXTNX'].rolling(window=13).mean()
mkt_data['VIXTNX25day_MA'] = mkt_data['VIXTNX'].rolling(window=25).mean()
mkt_data['VIXTNX50day_MA'] = mkt_data['VIXTNX'].rolling(window=50).mean()
mkt_data['VIXTNX100day_MA'] = mkt_data['VIXTNX'].rolling(window=100).mean()
mkt_data['VIXTNX150day_MA'] = mkt_data['VIXTNX'].rolling(window=150).mean()
mkt_data['VIXTNX200day_MA'] = mkt_data['VIXTNX'].rolling(window=200).mean()


mkt_data['VIXTNX20stdev']=mkt_data['VIXTNX'].rolling(20).std()
mkt_data['VIXTNXbbwidth']=mkt_data['VIXTNX20stdev']*4
mkt_data['VIXTNX20dMA']= mkt_data['VIXTNX'].rolling(20).mean()
mkt_data['VIXTNXbb%']= (mkt_data['VIXTNX']- mkt_data['VIXTNX20dMA'] + 2 * mkt_data['VIXTNX20stdev'])/(4 * mkt_data['VIXTNX20stdev'])
mkt_data['VIXTNXupbb']= mkt_data['VIXTNX20dMA']+2*mkt_data['VIXTNX20stdev']
mkt_data['VIXTNXlwbb']= mkt_data['VIXTNX20dMA']-2*mkt_data['VIXTNX20stdev']

mkt_data.tail()

,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,SPY_7d%Chg,...,VIXTNX50day_MA,VIXTNX100day_MA,VIXTNX150day_MA,VIXTNX200day_MA,VIXTNX20stdev,VIXTNXbbwidth,VIXTNX20dMA,VIXTNXbb%,VIXTNXupbb,VIXTNXlwbb
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-04,328.070007,330.010010,327.720001,329.059998,62573200,0.015241,0.022783,0.004211,0.006638,0.000882,...,0.775564,0.809043,0.850173,0.812402,0.193821,0.775285,0.843009,0.704104,1.230651,0.455366
2020-02-05,332.269989,333.089996,330.670013,332.859985,65951100,0.011548,0.026965,0.034594,0.019105,0.028933,...,0.779120,0.810291,0.851936,0.814598,0.193360,0.773441,0.851206,0.587314,1.237927,0.464486
2020-02-06,333.910004,334.190002,332.799988,333.980011,50359700,0.003365,0.014952,0.030421,0.019226,0.021689,...,0.783407,0.812170,0.853761,0.816759,0.191141,0.764564,0.860819,0.564294,1.243101,0.478537
2020-02-07,332.820007,333.989990,331.600006,332.200012,64088400,-0.005330,-0.001983,0.009542,0.032543,0.017084,...,0.789556,0.814005,0.855974,0.819056,0.187677,0.750706,0.876091,0.638888,1.251444,0.500738
2020-02-10,331.230011,334.750000,331.190002,334.679993,36449231,0.007465,0.002096,0.005468,0.032581,0.021362,...,0.795736,0.815758,0.857880,0.821302,0.183409,0.733635,0.890290,0.611655,1.257108,0.523473


In [36]:
nyadv = quandl.get("URC/NYSE_ADV", authtoken="LBfYCzzUxqdWE-P5gM5T")
nydec = quandl.get("URC/NYSE_DEC", authtoken="LBfYCzzUxqdWE-P5gM5T")
nyadu = quandl.get("URC/NYSE_UNCH", authtoken="LBfYCzzUxqdWE-P5gM5T")
nydnvol = quandl.get("URC/NYSE_DEC_VOL", authtoken="LBfYCzzUxqdWE-P5gM5T")
nyupvol = quandl.get("URC/NYSE_ADV_VOL", authtoken="LBfYCzzUxqdWE-P5gM5T")
nyunvol = quandl.get("URC/NYSE_UNCH_VOL", authtoken="LBfYCzzUxqdWE-P5gM5T")
ny52hi = quandl.get("URC/NYSE_52W_HI", authtoken="LBfYCzzUxqdWE-P5gM5T")
ny52lo = quandl.get("URC/NYSE_52W_LO", authtoken="LBfYCzzUxqdWE-P5gM5T")
sent = quandl.get("AAII/AAII_SENTIMENT", authtoken="LBfYCzzUxqdWE-P5gM5T")
pmi = quandl.get("ISM/MAN_PMI", authtoken="LBfYCzzUxqdWE-P5gM5T")

In [37]:
nyadv.rename(columns={'Date': 'Date', 'Numbers of Stocks': 'NYADV'}, inplace = True )
nydec.rename(columns ={'Date':'Date', 'Numbers of Stocks': "NYDEC"}, inplace = True)
nyadu.rename(columns={'Date':'Date', 'Numbers of Stocks': 'NYADU'}, inplace=True)
nydnvol.rename(columns={'Date':'Date', 'Numbers of Stocks': 'NYDNVOL'}, inplace=True)
nyupvol.rename(columns={'Date':'Date', 'Numbers of Stocks': 'NYUPVOL'}, inplace=True)
nyunvol.rename(columns={'Date':'Date', 'Numbers of Stocks': 'NYUNVOL'}, inplace=True)
ny52hi.rename(columns={'Date':'Date', 'Numbers of Stocks': 'NY52HI'}, inplace=True)
ny52lo.rename(columns={'Date':'Date', 'Numbers of Stocks': 'NY52LO'}, inplace=True)
fedhold.rename(columns={'Date':'Date', 'WSHOSHO':'FedHldBal'}, inplace=True)
fedbal.rename(columns={'Date':'Date', 'WALCL':'FedTotBS'}, inplace=True)
stres.rename(columns={'Date':'Date', 'STLFSI':'FedStres'},inplace=True)
freight.rename(columns={'Date':'Date','FRGSHPUSM649NCIS':'Freight'},inplace=True)
leading.rename(columns={'Date':'Date','USALOLITONOSTSAM':'LeadEcoInc'},inplace=True)
hyspread.rename(columns={'Date':'Date','BAMLH0A0HYM2': 'High Yield Spread'}, inplace=True)
tentwospr.rename(columns={'Date':'Date','T10Y2Y':'10-2 Treas Spr'}, inplace=True)
fedrecprob.rename(columns={'Date':'Date','RECPROUSM156N':'Fed Recession Prob'}, inplace=True)


In [38]:
leading.head()

,LeadEcoInc
DATE,
2003-01-01,97.874659
2003-02-01,97.846193
2003-03-01,97.889520
2003-04-01,98.027059
2003-05-01,98.249626


In [39]:
sent.drop(['Neutral', 'Total','Bullish 8-Week Mov Avg','Bullish Average + St. Dev','Bullish Average','Bullish Average - St. Dev','S&P 500 Weekly High','S&P 500 Weekly Low','S&P 500 Weekly Close'], axis=1,inplace=True)

In [40]:
sent.rename(columns={'Bullish':'Bullish Sentiment', 'Bearish':'Bearish Sentiment', 'Bull-Bear Spread':'Bull-Bear Sentiment Spread'})

,Bullish Sentiment,Bearish Sentiment,Bull-Bear Sentiment Spread
Date,,,
1987-06-26,NaN,NaN,NaN
1987-07-17,NaN,NaN,NaN
1987-07-24,0.360000,0.140000,0.220000
1987-07-31,0.260000,0.260000,0.000000
1987-08-07,0.560000,0.290000,0.270000
...,...,...,...
2020-01-09,0.330688,0.298942,0.031746
2020-01-16,0.418338,0.275072,0.143266
2020-01-23,0.456019,0.247685,0.208334


In [41]:

#combine all dataframes into one dataframe

nyse_data=mkt_data.join(nyadv).join(nydec).join(nyadu).join(nyupvol).join(nydnvol).join(nyunvol).join(ny52lo).join(ny52hi).join(fedhold).join(fedbal).join(sent).join(stres).join(freight).join(pmi).join(leading).join(hyspread).join(tentwospr).join(fedrecprob)

nyse_data.index.name='Date'
nyse_data.describe()

,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,SPY_7d%Chg,...,Bullish,Bearish,Bull-Bear Spread,FedStres,Freight,PMI,LeadEcoInc,High Yield Spread,10-2 Treas Spr,Fed Recession Prob
count,4267.000000,4267.000000,4267.000000,4267.000000,4.267000e+03,4266.000000,4265.000000,4264.000000,4262.000000,4260.000000,...,863.000000,863.000000,863.000000,855.000000,130.000000,130.000000,130.000000,4263.000000,4235.000000,130.000000
mean,145.241532,145.984314,144.424383,145.255980,1.290508e+08,0.000463,0.000918,0.001369,0.002272,0.003156,...,0.383794,0.323595,0.060200,-0.426298,1.140723,53.625385,99.908512,5.392179,1.356897,6.693846
std,68.302331,68.475679,68.093501,68.311107,9.779921e+07,0.011230,0.015223,0.017905,0.022490,0.025793,...,0.093739,0.091747,0.168299,1.004305,0.101956,4.510647,1.170111,2.777493,0.889881,22.257458
min,54.374998,56.015453,53.694809,54.503036,8.055800e+06,-0.098448,-0.133555,-0.133881,-0.197934,-0.237463,...,0.164800,0.085700,-0.513500,-1.564000,0.878000,33.100000,95.089491,2.410000,-0.190000,0.060000
25%,92.007624,92.371478,91.357303,92.026791,6.186080e+07,-0.003723,-0.005614,-0.006551,-0.007732,-0.008363,...,0.316557,0.259707,-0.049248,-1.100000,1.081750,51.125000,99.461143,3.750000,0.520000,0.160000
50%,114.054370,114.680999,113.190105,114.120750,9.884070e+07,0.000717,0.001581,0.002454,0.003832,0.005351,...,0.377400,0.311600,0.061800,-0.575000,1.142500,53.450000,100.105032,4.580000,1.440000,0.250000
75%,190.600012,191.639437,189.693582,190.705048,1.654109e+08,0.005418,0.008382,0.010481,0.014071,0.017073,...,0.439119,0.382332,0.161147,-0.216500,1.206000,56.675000,100.637122,6.180000,2.165000,0.535000
max,333.910004,334.750000,332.799988,334.679993,8.710263e+08,0.145198,0.128249,0.147112,0.194035,0.173543,...,0.714300,0.702700,0.628600,5.265000,1.347000,60.800000,101.809742,21.820000,2.910000,99.800000


In [42]:
nyse_data = nyse_data.replace(0, np.nan).ffill()


In [43]:
list(nyse_data.columns)

['SPY_Open',
 'SPY_High',
 'SPY_Low',
 'SPY_Close',
 'SPY_Volume',
 'SPY_1d%Chg',
 'SPY_2d%Chg',
 'SPY_3d%Chg',
 'SPY_5d%Chg',
 'SPY_7d%Chg',
 'SPY_10d%Chg',
 'SPY_14d%Chg',
 'SPY_21d%Chg',
 'SPY_30d%Chg',
 'SPY_45d%Chg',
 'SPY_60d%Chg',
 'SPY_90d%Chg',
 'SPY_120d%Chg',
 'SPY_180d%Chg',
 'SPY_252d%Chg',
 'SPY_200day_MA',
 'SPY_150day_MA',
 'SPY_100day_MA',
 'SPY_50day_MA',
 'SPY_25day_MA',
 'SPY_13day_MA',
 'SPY_8day_MA',
 'SPY_5day_MA',
 'SPY_20stdev',
 'SPY_bbwidth',
 'SPY_20dMA',
 'SPY_bb%',
 'SPY_upbb',
 'SPY_lwbb',
 'TLT_Open',
 'TLT_High',
 'TLT_Low',
 'TLT_Close',
 'TLT_Volume',
 'TLT_1d%Chg',
 'TLT_2d%Chg',
 'TLT_3d%Chg',
 'TLT_5d%Chg',
 'TLT_7d%Chg',
 'TLT_10d%Chg',
 'TLT_14d%Chg',
 'TLT_21d%Chg',
 'TLT_30d%Chg',
 'TLT_45d%Chg',
 'TLT_60d%Chg',
 'TLT_90d%Chg',
 'TLT_120d%Chg',
 'TLT_180d%Chg',
 'TLT_252d%Chg',
 'TLT_200day_MA',
 'TLT_150day_MA',
 'TLT_100day_MA',
 'TLT_50day_MA',
 'TLT_25day_MA',
 'TLT_13day_MA',
 'TLT_8day_MA',
 'TLT_5day_MA',
 'TLT_20stdev',
 'TLT_bbwidth',


In [44]:
nyse_data.isnull()

,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,SPY_7d%Chg,...,Bullish,Bearish,Bull-Bear Spread,FedStres,Freight,PMI,LeadEcoInc,High Yield Spread,10-2 Treas Spr,Fed Recession Prob
Date,,,,,,,,,,,,,,,,,,,,,
2003-02-28,False,False,False,False,False,True,True,True,True,True,...,True,True,True,False,True,True,True,False,False,True
2003-03-03,False,False,False,False,False,False,True,True,True,True,...,True,True,True,False,True,True,True,False,False,True
2003-03-04,False,False,False,False,False,False,False,True,True,True,...,True,True,True,False,True,True,True,False,False,True
2003-03-05,False,False,False,False,False,False,False,False,True,True,...,True,True,True,False,True,True,True,False,False,True
2003-03-06,False,False,False,False,False,False,False,False,True,True,...,False,False,False,False,True,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-02-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-02-06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [45]:
nyse_data.reset_index(level=0, inplace=True)

In [46]:
# Setting target to predict 3 day future performance of SPY

nyse_data['Target']=(nyse_data['SPY_Close'].shift(-3)-nyse_data['SPY_Close'])/nyse_data['SPY_Close']
nyse_data.tail()

,Date,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,...,Bearish,Bull-Bear Spread,FedStres,Freight,PMI,LeadEcoInc,High Yield Spread,10-2 Treas Spr,Fed Recession Prob,Target
4262,2020-02-04,328.070007,330.010010,327.720001,329.059998,62573200,0.015241,0.022783,0.004211,0.006638,...,0.368564,-0.048781,-1.463,1.124,48.1,98.993438,3.85,0.20,0.52,0.009542
4263,2020-02-05,332.269989,333.089996,330.670013,332.859985,65951100,0.011548,0.026965,0.034594,0.019105,...,0.368564,-0.048781,-1.463,1.124,48.1,98.993438,3.75,0.22,0.52,0.005468
4264,2020-02-06,333.910004,334.190002,332.799988,333.980011,50359700,0.003365,0.014952,0.030421,0.019226,...,0.352151,-0.013441,-1.463,1.124,48.1,98.993438,3.70,0.21,0.52,NaN
4265,2020-02-07,332.820007,333.989990,331.600006,332.200012,64088400,-0.005330,-0.001983,0.009542,0.032543,...,0.352151,-0.013441,-1.463,1.124,48.1,98.993438,3.75,0.18,0.52,NaN
4266,2020-02-10,331.230011,334.750000,331.190002,334.679993,36449231,0.007465,0.002096,0.005468,0.032581,...,0.352151,-0.013441,-1.463,1.124,48.1,98.993438,3.75,0.19,0.52,NaN


In [47]:
nyse_data['Target']= np.sign(nyse_data['Target'])
nyse_data.head(10)

,Date,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,...,Bearish,Bull-Bear Spread,FedStres,Freight,PMI,LeadEcoInc,High Yield Spread,10-2 Treas Spr,Fed Recession Prob,Target
0,2003-02-28,60.370961,60.914137,60.149405,60.678284,43666800,NaN,NaN,NaN,NaN,...,NaN,NaN,0.461,NaN,NaN,NaN,8.38,2.18,NaN,-1.0
1,2003-03-03,60.935599,61.307242,59.834955,60.099392,42923100,-0.009540,NaN,NaN,NaN,...,NaN,NaN,0.461,NaN,NaN,NaN,8.29,2.16,NaN,-1.0
2,2003-03-04,59.999304,60.042190,59.070193,59.141663,31440500,-0.015936,-0.025324,NaN,NaN,...,NaN,NaN,0.461,NaN,NaN,NaN,8.31,2.16,NaN,1.0
3,2003-03-05,59.041627,59.706302,58.862952,59.641975,43974700,0.008460,-0.007611,-0.017079,NaN,...,NaN,NaN,0.461,NaN,NaN,NaN,8.33,2.17,NaN,-1.0
4,2003-03-06,59.227429,59.691981,58.941547,59.141663,41217700,-0.008389,-0.007611,-0.015936,NaN,...,0.3939,-0.0606,0.461,NaN,NaN,NaN,8.31,2.20,NaN,-1.0
5,2003-03-07,58.326938,60.027930,58.198291,59.549080,63538000,0.006889,-0.001558,0.006889,-0.018610,...,0.3939,-0.0606,0.422,NaN,NaN,NaN,8.34,2.24,NaN,-1.0
6,2003-03-10,59.034498,59.220322,57.962443,58.119678,41014100,-0.024004,-0.017280,-0.025524,-0.032941,...,0.3939,-0.0606,0.422,NaN,NaN,NaN,8.38,2.24,NaN,1.0
7,2003-03-11,58.234025,58.605668,57.519322,57.547905,48102700,-0.009838,-0.033605,-0.026948,-0.026948,...,0.3939,-0.0606,0.422,NaN,NaN,NaN,8.33,2.22,NaN,1.0
8,2003-03-12,57.447843,57.962430,56.733140,57.933842,62459800,0.006706,-0.003197,-0.027124,-0.028640,...,0.3939,-0.0606,0.422,NaN,NaN,NaN,8.32,2.15,NaN,1.0
9,2003-03-13,58.734300,59.970739,58.269742,59.935001,72117800,0.034542,0.041480,0.031234,0.013414,...,0.5143,-0.1714,0.422,NaN,NaN,NaN,8.16,2.15,NaN,1.0


In [48]:
# for simplicity in later sections I'm going to make a change to our prediction column. To keep this as a binary classifier I'll change flat days and consider them up days.
nyse_data.loc[nyse_data['Target'] == 0.0] = 1.0

In [49]:
nyse_data.isnull().sum()

Date                   0
SPY_Open               0
SPY_High               0
SPY_Low                0
SPY_Close              0
                      ..
LeadEcoInc            22
High Yield Spread      0
10-2 Treas Spr         0
Fed Recession Prob    22
Target                 3
Length: 201, dtype: int64

In [50]:
 # We need to remove all rows that have an NaN value.
print('Before NaN Drop we have {} rows and {} columns'.format(nyse_data.shape[0], nyse_data.shape[1]))

# Any row that has a `NaN` value will be dropped.
nyse_data = nyse_data.dropna()

# Display how much we have left now.
print('After NaN Drop we have {} rows and {} columns'.format(nyse_data.shape[0], nyse_data.shape[1]))

# Print the head.
nyse_data.head()

Before NaN Drop we have 4267 rows and 201 columns
After NaN Drop we have 3595 rows and 201 columns


,Date,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_1d%Chg,SPY_2d%Chg,SPY_3d%Chg,SPY_5d%Chg,...,Bearish,Bull-Bear Spread,FedStres,Freight,PMI,LeadEcoInc,High Yield Spread,10-2 Treas Spr,Fed Recession Prob,Target
191,1,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,...,1.0000,1.000,1.000,1.000,1.0,1.000000,1.00,1.00,1.00,1.0
381,1,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,...,1.0000,1.000,1.000,1.000,1.0,1.000000,1.00,1.00,1.00,1.0
446,1,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,...,1.0000,1.000,1.000,1.000,1.0,1.000000,1.00,1.00,1.00,1.0
672,2005-10-27 00:00:00,89.322622,89.450016,88.370949,88.498337,66623100.0,-0.010639,-0.013532,-0.015505,0.003654,...,0.4615,-0.141,-0.309,1.277,56.8,100.420899,3.57,0.21,9.74,1.0
673,2005-10-28 00:00:00,88.745612,89.884622,88.498325,89.772224,72322000.0,0.014394,0.003602,0.000668,0.014137,...,0.4615,-0.141,-0.298,1.277,56.8,100.420899,3.58,0.18,9.74,1.0


In [51]:
list(nyse_data.columns)

['Date',
 'SPY_Open',
 'SPY_High',
 'SPY_Low',
 'SPY_Close',
 'SPY_Volume',
 'SPY_1d%Chg',
 'SPY_2d%Chg',
 'SPY_3d%Chg',
 'SPY_5d%Chg',
 'SPY_7d%Chg',
 'SPY_10d%Chg',
 'SPY_14d%Chg',
 'SPY_21d%Chg',
 'SPY_30d%Chg',
 'SPY_45d%Chg',
 'SPY_60d%Chg',
 'SPY_90d%Chg',
 'SPY_120d%Chg',
 'SPY_180d%Chg',
 'SPY_252d%Chg',
 'SPY_200day_MA',
 'SPY_150day_MA',
 'SPY_100day_MA',
 'SPY_50day_MA',
 'SPY_25day_MA',
 'SPY_13day_MA',
 'SPY_8day_MA',
 'SPY_5day_MA',
 'SPY_20stdev',
 'SPY_bbwidth',
 'SPY_20dMA',
 'SPY_bb%',
 'SPY_upbb',
 'SPY_lwbb',
 'TLT_Open',
 'TLT_High',
 'TLT_Low',
 'TLT_Close',
 'TLT_Volume',
 'TLT_1d%Chg',
 'TLT_2d%Chg',
 'TLT_3d%Chg',
 'TLT_5d%Chg',
 'TLT_7d%Chg',
 'TLT_10d%Chg',
 'TLT_14d%Chg',
 'TLT_21d%Chg',
 'TLT_30d%Chg',
 'TLT_45d%Chg',
 'TLT_60d%Chg',
 'TLT_90d%Chg',
 'TLT_120d%Chg',
 'TLT_180d%Chg',
 'TLT_252d%Chg',
 'TLT_200day_MA',
 'TLT_150day_MA',
 'TLT_100day_MA',
 'TLT_50day_MA',
 'TLT_25day_MA',
 'TLT_13day_MA',
 'TLT_8day_MA',
 'TLT_5day_MA',
 'TLT_20stdev',
 'TLT_b

In [52]:
import os
import sys
import requests

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree
import seaborn as sns
import graphviz
import pydotplus
import io
from scipy import misc

In [54]:
# Grab our X & Y Columns.
X_Cols = nyse_data[['SPY_Close',
 'SPY_Volume',
 'SPY_1d%Chg',
 'SPY_2d%Chg',
 'SPY_3d%Chg',
 'SPY_5d%Chg',
 'SPY_7d%Chg',
 'SPY_10d%Chg',
 'SPY_14d%Chg',
 'SPY_21d%Chg',
 'SPY_30d%Chg',
 'SPY_45d%Chg',
 'SPY_60d%Chg',
 'SPY_90d%Chg',
 'SPY_120d%Chg',
 'SPY_180d%Chg',
 'SPY_252d%Chg',
 'SPY_200day_MA',
 'SPY_150day_MA',
 'SPY_100day_MA',
 'SPY_50day_MA',
 'SPY_25day_MA',
 'SPY_13day_MA',
 'SPY_8day_MA',
 'SPY_5day_MA',
 'SPY_20stdev',
 'SPY_bbwidth',
 'SPY_20dMA',
 'SPY_bb%',
 'SPY_upbb',
 'SPY_lwbb',
 'TLT_Close',
 'TLT_Volume',
 'TLT_1d%Chg',
 'TLT_2d%Chg',
 'TLT_3d%Chg',
 'TLT_5d%Chg',
 'TLT_7d%Chg',
 'TLT_10d%Chg',
 'TLT_14d%Chg',
 'TLT_21d%Chg',
 'TLT_30d%Chg',
 'TLT_45d%Chg',
 'TLT_60d%Chg',
 'TLT_90d%Chg',
 'TLT_120d%Chg',
 'TLT_180d%Chg',
 'TLT_252d%Chg',
 'TLT_200day_MA',
 'TLT_150day_MA',
 'TLT_100day_MA',
 'TLT_50day_MA',
 'TLT_25day_MA',
 'TLT_13day_MA',
 'TLT_8day_MA',
 'TLT_5day_MA',
 'TLT_20stdev',
 'TLT_bbwidth',
 'TLT_20dMA',
 'TLT_bb%',
 'TLT_upbb',
 'TLT_lwbb',
 'VIX_Open',
 'VIX_High',
 'VIX_Low',
 'VIX_Close',
 'VIX_1d%Chg',
 'VIX_2d%Chg',
 'VIX_3d%Chg',
 'VIX_5d%Chg',
 'VIX_7d%Chg',
 'VIX_10d%Chg',
 'VIX_14d%Chg',
 'VIX_21d%Chg',
 'VIX_30d%Chg',
 'VIX_45d%Chg',
 'VIX_60d%Chg',
 'VIX_90d%Chg',
 'VIX_120d%Chg',
 'VIX_180d%Chg',
 'VIX_252d%Chg',
 'VIX_200day_MA',
 'VIX_150day_MA',
 'VIX_100day_MA',
 'VIX_50day_MA',
 'VIX_25day_MA',
 'VIX_13day_MA',
 'VIX_8day_MA',
 'VIX_5day_MA',
 'VIX_20stdev',
 'VIX_bbwidth',
 'VIX_20dMA',
 'VIX_bb%',
 'VIX_upbb',
 'VIX_lwbb',
 'TNX_Close',
 'TNX_1d%Chg',
 'TNX_2d%Chg',
 'TNX_3d%Chg',
 'TNX_5d%Chg',
 'TNX_7d%Chg',
 'TNX_10d%Chg',
 'TNX_14d%Chg',
 'TNX_21d%Chg',
 'TNX_30d%Chg',
 'TNX_45d%Chg',
 'TNX_60d%Chg',
 'TNX_90d%Chg',
 'TNX_120d%Chg',
 'TNX_180d%Chg',
 'TNX_252d%Chg',
 'TNX_200day_MA',
 'TNX_150day_MA',
 'TNX_100day_MA',
 'TNX_50day_MA',
 'TNX_25day_MA',
 'TNX_13day_MA',
 'TNX_8day_MA',
 'TNX_5day_MA',
 'stkbdDiff',
 'stkbdDiff3d',
 'stkbdDiff5d',
 'stkbdDiff7d',
 'stkbdDiff10d',
 'stkbdDiff14d',
 'stkbdDiff21d',
 'stkbdDiff30d',
 'stkbdDiff45d',
 'stkbdDiff60d',
 'stkbdDiff90d',
 'stkbdDiff120d',
 'stkbdDiff180d',
 'stkbdDiff252d',
 'diff5day_MA',
 'diff8day_MA',
 'diff13day_MA',
 'diff25day_MA',
 'diff50day_MA',
 'diff100day_MA',
 'diff150day_MA',
 'diff200day_MA',
 'VIXTNX',
 'VIXTNX1d%Chg',
 'VIXTNX2d%Chg',
 'VIXTNX3d%Chg',
 'VIXTNX5d%Chg',
 'VIXTNX7d%Chg',
 'VIXTNX10d%Chg',
 'VIXTNX14d%Chg',
 'VIXTNX21d%Chg',
 'VIXTNX30d%Chg',
 'VIXTNX60d%Chg',
 'VIXTNX90d%Chg',
 'VIXTNX120d%Chg',
 'VIXTNX180d%Chg',
 'VIXTNX252d%Chg',
 'VIXTNX5day_MA',
 'VIXTNX8day_MA',
 'VIXTNX13day_MA',
 'VIXTNX25day_MA',
 'VIXTNX50day_MA',
 'VIXTNX100day_MA',
 'VIXTNX150day_MA',
 'VIXTNX200day_MA',
 'VIXTNX20stdev',
 'VIXTNXbbwidth',
 'VIXTNX20dMA',
 'VIXTNXbb%',
 'VIXTNXupbb',
 'VIXTNXlwbb',
 'NYADV',
 'NYDEC',
 'NYADU',
 'NYUPVOL',
 'NYDNVOL',
 'NYUNVOL',
 'NY52LO',
 'NY52HI',
 'FedHldBal',
 'FedTotBS',
 'Bullish',
 'Bearish',
 'Bull-Bear Spread',
 'FedStres',
 'Freight',
 'PMI',
 'LeadEcoInc',
 'High Yield Spread',
 '10-2 Treas Spr',
 'Fed Recession Prob',]]

Y_Cols = nyse_data['Target']

# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X_Cols, Y_Cols, random_state = 0)

# Create a Random Forest Classifier
rand_frst_clf = RandomForestClassifier(n_estimators = 200, oob_score = True, criterion = "gini", random_state = 0)

# Fit the data to the model
rand_frst_clf.fit(X_train, y_train)

# Make predictions
y_pred = rand_frst_clf.predict(X_test)

In [55]:
# Print the Accuracy of our Model.
print('Correct Prediction (%): ', accuracy_score(y_test, rand_frst_clf.predict(X_test), normalize = True) * 100.0)

Correct Prediction (%):  73.85984427141268


In [56]:
#find feature importance using Random Forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=50, max_depth=4, min_samples_leaf=4, max_features=0.1)
rf.fit(X_Cols, Y_Cols)
features =X_Cols.columns.values